# Chapter 1 The Learning Problem

## Exercises

#### Exercise 1.1

1. (a) 
  * Input space $\mathcal{X}$: patient's medical history, symptoms, personal health information etc.
  * Output space $\mathcal{Y}$: all possible diseases
  * target function $f: \mathcal{X} \to \mathcal{Y}$: ideal formula to identify a patient's problem
  * Data set: All available patients' information and their corresponding correct problem diagnostic.
2. (b)
  * Input space $\mathcal{X}$: handwritten digits (digitalized)
  * Output space $\mathcal{Y}$: 0-9 digits
  * target function $f: \mathcal{X} \to \mathcal{Y}$: ideal formula match a handwritten digit to a correct digit
  * Data set: handwritten digits and their corresponding correct matches
3. (c)
  * Input space $\mathcal{X}$: every information of an email, e.g. words
  * Output space $\mathcal{Y}$: yes/no 
  * target function $f: \mathcal{X} \to \mathcal{Y}$: ideal formula to identify whether an email is spam or not
  * Data set: Spam and non-spam emails that have been identified by human
4. (d)
  * Input space $\mathcal{X}$: price of electric, temperature, day of the week
  * Output space $\mathcal{Y}$: electric load
  * target function $f: \mathcal{X} \to \mathcal{Y}$: ideal function that gives exact electric load for a given price, temperature and day of the week.
  * Data set: historical electric load along with corresponding price, temperature and day of the week information.
5. (e) TODO
  * Input space $\mathcal{X}$: 
  * Output space $\mathcal{Y}$: 
  * target function $f: \mathcal{X} \to \mathcal{Y}$: 
  * Data set: 

#### Exercise 1.2
1. (a) Keywords with a large positive weight: free, cheap, earn, !
1. (b) Keywords with a negative weight: person name, hi, the
1. (c) The parameter $b$ in perceptron directly affects how many borderline messages end up being classified as spam. This is because $b$ is the threshold used to classify the emails into spam and non-spam categories.

#### Exercise 1.3
1. (a) If $x(t)$ is misclassified by $w(t)$, then $w^T(t)x(t)$ has different signs of $y(t)$, thus $y(t)w^T(t)x(t) \gt 0$.
2. (b) 
\begin{align}
y(t)w^T(t+1)x(t)  &= y(t) \left(w(t)+y(t)x(t)\right)^Tx(t) \\
&= y(t)\left(w^T(t) + y(t)x^T(t)\right)x(t) \\
&= y(t)w^T(t)x(t) + y(t)y(t)x^T(t)x(t)\\
&\gt y(t)w^T(t)x(t) \;\;\;\text{because the last term is } \ge \text{ than } 0\\
\end{align}
3. (c) 

From previous problem, we see that $y(t)w^T(t)x(t)$ is increasing with each update. If $y(t)$ is positive, but $w^T(t)x(t)$ is negative, we move $w^T(t)x(t)$ toward positive by increasing it. If however $y(t)$ is negative, but $w^T(t)x(t)$ is positive, $y(t)w^T(t)x(t)$ increases means $w^T(t)x(t)$ is decreasing, i.e. moving toward negative region. 

So the move from $w(t)$ to $w(t+1)$ is a move "in the right direction" as far as classifying $x(t)$ is concerned.

#### Exercise 1.4 TODO
Implement the perceptron learning algorithm and check
* Convergence speed
* How well the final hypothesis $g$ matches your target $f$

#### Exercise 1.5
1. (a) Learning
1. (b) Design
1. (c) Learning
1. (d) Design
1. (e) Learning

#### Exercise 1.6
1. (a) Supervised Learning
1. (b) Reinforcement Learning
1. (c) Unsupervised Learning
1. (d) Learning to play music
  * If learn by yourself, it's unsupervised learning
  * If learn from a teacher, it's supervised learning
  * If learn by yourself but with someone to tell you if your music is good or not, it's reinforcement learning. 
1. (e) Supervised Learning

#### Exercise 1.7
1. (a) The learning algorithm will pick the final hypothesis that always returns 1. 
  * 1 out of 8 $f$ agrees with $g$ on all three points; 3 $f$ agree with $g$ on two of the points; 3 $f$ agree with $g$ on one of the points, 1 $f$ agrees with none of the points. 
1. (b) The learning algorithm will pick the final hypothesis that always returns 0. 
  * 1 out of 8 $f$ agrees with $g$ on all three points; 3 $f$ agree with $g$ on two of the points; 3 $f$ agree with $g$ on one of the points, 1 $f$ agrees with none of the points.
1. (c) The learning algorithm will pick the final hypothesis $XOR$. 
  * 1 out of 8 $f$ agrees with $g$ on all three points; 3 $f$ agree with $g$ on two of the points; 3 $f$ agree with $g$ on one of the points, 1 $f$ agrees with none of the points.
1. (d) The learning algorithm will pick the final hypothesis $f_7$. 
  * 1 out of 8 $f$ agrees with $g$ on all three points; 3 $f$ agree with $g$ on two of the points; 3 $f$ agree with $g$ on one of the points, 1 $f$ agrees with none of the points.
  
#### Exercise 1.8

In a sample of 10 marbles, for the fraction $\nu$ of red marbles to be $\nu \le 0.1$, we must have at most one red marbles. 

\begin{align}
P(number\;of\;red \le 1) &= P(red = 0) + P(red = 1) \\
&= (1-\mu)^{10} + \mu (1-\mu)^9\\
&= (1-\mu)^9\\
&= 1.0e-9
\end{align}

#### Exercise 1.9
We have $\mu=0.9$, $N=10$, and want $\nu \le 0.1$, i.e. $|\mu - \nu| = \mu - \nu \ge 0.9 - 0.1 = 0.8$. Let's pick $\epsilon = 0.7$, then according to Hoeffding Inequity, we have

\begin{align}
P(\nu \le 0.1) &= P(\mu - \nu \ge 0.8)\\
&= P(|\mu - \nu| \ge 0.8) \\
&\le P(|\mu - \nu| \gt 0.7) \\
&= P(|\mu - \nu| \gt \epsilon) \\
&\le 2e^{-2\epsilon^2N}\\
&\approx 0.0001109032
\end{align}

This is an upper bound of the probability from pervious problem and is much larger than the calculated probability.

#### Exercise 1.10 TODO
A compute simulation exercise.

#### Exercise 1.11

1. (a) $S$ can not produce a hypothesis that is guaranteed to perform better than random on any point outside $\mathcal{D}$. 
If $f$ has 25 $+1$ on $\mathcal{D}$ but $-1$ on all other points in $\mathcal{X}$, $S$ will choose the hypothesis $h_1$, which will not match $f$ outside of $\mathcal{D}$ at all. On the other hand, a random function will have $+1$ and $-1$ 50/50, and it matches $f$ half of time, which is better than the function produced by $S$.

1. (b) It is possible that $C$ produces a better hypothesis than $S$ produces. See the example above.
1. (c) If every point in $\mathcal{D}$ has 1, then $S$ will choose $h_1$ and $C$ will choose $h_2$. So outside of $\mathcal{D}$, $h_1$ will have 90% chance to match with $f$, while $h_2$ will have only 10% chance. $S$ will always produce a better hypothesis than $C$. 

1. (d) From previous problem, we can see that when $p \lt 0.5$, $C$ will produce a better hypothesis than $S$. Since $C$ always produce $h_2$, which will match $f$ better than $h_1$ if $p \lt 0.5$.

#### Exercise 1.12

I think the best I can promise is (c). 
* The unknown target $f$ can be very complex that we can't learn at all.
* If we can learn and produce a hypothesis $g$, since there are many data points (4000), the probability that $g$ matches $f$ is high according to Hoeffding inequality, and the error on $g$ might be small since we have a large data set.

#### Exercise 1.13

1. (a) The probability of error that $h$ makes in approximating $y$ is

\begin{align}
P(h \ne y) &= P(h \ne y| y = f(x))P(y=f(x)) + P(h\ne y| y \ne f(x))P(y\ne f(x))\\
&= \mu \lambda + (1-\mu)(1-\lambda)\\
&= \mu (2 \lambda - 1) + (1-\lambda)\\
\end{align}

1. (b) It can be seen from previous problem, that when $\lambda = 0.5$, $P(h\ne y) = 1-\lambda = 0.5$, is independent of $\mu$.

## Problems

#### Problem 1.1

\begin{align}
P(1st \;ball \;is\; black) &= P(1st\; ball\; is\; black|picked\; black,black\; bag)P(picked\; black,black\; bag) + P(1st\; ball\; is\; black|picked\; black,white\; bag)P(picked\; black,white\; bag)\\
&= 1.0*0.5 + 0.5 * 0.5\\
&= 0.75\\
\end{align}

\begin{align}
P(picked\;black,black \;bag|1st\; ball\; is\; black) &= \frac{P(picked\; black,black\; bag\; and\; 1st\; ball\; is\; black)}{P(1st\; ball\; is\; black)} \\
&= \frac{P(1st\; ball\; is\; black|picked\; black,black\; bag)P(picked\; black,black\; bag)}{P(1st \;ball \;is\; black)}\\
&= \frac{1.0*0.5}{0.75}\\
&= \frac{2}{3}\\
\end{align}

\begin{align}
P(picked\;black,white \;bag|1st\; ball\; is\; black) &= \frac{1}{3}\\
\end{align}

Now we can compute the probability that the second ball is also black:

\begin{align}
P(2nd\; ball\; is \;black) &= P(2nd\; ball\; is\; black|picked\;black,black \;bag|1st\; ball\; is\; black)*P(picked\;black,black \;bag|1st\; ball\; is\; black) + P(2nd\; ball\; is \;black|picked\;black,white \;bag|1st\; ball\; is\; black)*P(picked\;black,white \;bag|1st\; ball\; is\; black)\\
&= 1.0*\frac{2}{3} + 0*\frac{1}{3}\\
&= \frac{2}{3}
\end{align}

#### Problem 1.2

